# 1 - Importação e limpeza preliminar dos dados

In [1]:
import pickle

import numpy as np
import pandas as pd
from scipy import stats as spst
from tqdm import tqdm

## Importação

In [2]:
covid_raw = pd.read_csv(r'../data/COVID.csv', index_col = 'Unnamed: 0')
covid_raw.index.name = 'id'

covid_raw.head()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
0,0,1,NaN,0.0,27,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN
1,0,1,NaN,0.0,24,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,NaN
2,1,0,0.0,0.0,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,1,0.0
3,0,0,0.0,1.0,30,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,0.0
4,1,0,0.0,0.0,60,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN,1,0.0


## Limpeza preliminar

Vamos fazer uma limpeza preliminar, antes de explorar os dados mais a fundo.

Os dados criados aqui serão exportados para um objeto `pickle`. Esse objeto `pickle` será importado nos *notebooks*  subsequentes para análise.

### Alteração de tipos

In [3]:
covid_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499692 entries, 0 to 499691
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sex                  499692 non-null  int64  
 1   patient_type         499692 non-null  int64  
 2   intubed              107424 non-null  float64
 3   pneumonia            499681 non-null  float64
 4   age                  499692 non-null  int64  
 5   pregnancy            245258 non-null  float64
 6   diabetes             498051 non-null  float64
 7   copd                 498246 non-null  float64
 8   asthma               498250 non-null  float64
 9   inmsupr              498030 non-null  float64
 10  hypertension         498203 non-null  float64
 11  other_disease        497499 non-null  float64
 12  cardiovascular       498183 non-null  float64
 13  obesity              498222 non-null  float64
 14  renal_chronic        498216 non-null  float64
 15  tobacco          

Gastamos 76 MB de memória para armazenar todo o conjunto.

In [4]:
def acertar_tipos(df):
    df = df.copy()

    df = df.astype(pd.Int8Dtype())  # porque não 'int64' normal do numpy? Porque o 'Int' do pandas acomoda a presença de NaNs (o do numpy não); o '8' é só para economizar memória (e tornar a análise mais ágil)
    cols_minus_age = list(set(df.columns) - set(['age']))
    df[cols_minus_age] = df[cols_minus_age].astype('category')

    return df

covid_t1 = (covid_raw
    .pipe(acertar_tipos)
)

In [5]:
covid_t1.head()

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
0,0,1,NaN,0,27,NaN,0,0,0,0,0,0,0,0,0,0,0,1,NaN
1,0,1,NaN,0,24,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
2,1,0,0,0,54,0,0,0,0,0,0,0,0,1,0,0,NaN,1,0
3,0,0,0,1,30,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,0
4,1,0,0,0,60,0,1,0,0,0,1,0,1,0,0,0,NaN,1,0


In [6]:
covid_t1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499692 entries, 0 to 499691
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   sex                  499692 non-null  category
 1   patient_type         499692 non-null  category
 2   intubed              107424 non-null  category
 3   pneumonia            499681 non-null  category
 4   age                  499692 non-null  Int8    
 5   pregnancy            245258 non-null  category
 6   diabetes             498051 non-null  category
 7   copd                 498246 non-null  category
 8   asthma               498250 non-null  category
 9   inmsupr              498030 non-null  category
 10  hypertension         498203 non-null  category
 11  other_disease        497499 non-null  category
 12  cardiovascular       498183 non-null  category
 13  obesity              498222 non-null  category
 14  renal_chronic        498216 non-null  category
 15  

Como convertemos os tipos de `float` (visto que há `NaN`s espalhados pelas colunas, o que força o `pandas` a usar o `float` como o tipo das colunas) para `category`, agora o conjunto requer 13 MB, uma redução de **82.5%**.

### Análise de *outliers* preliminar

Vamos ver se há algum *outlier*, ou seja, algum dado que claramente não é correto.

#### Idades

In [7]:
covid_t1.describe()

,age
count,499692.000000
mean,42.545942
std,16.640391
min,0.000000
25%,31.000000
50%,41.000000
75%,53.000000
max,120.000000


In [8]:
covid_t1[covid_t1.age > 100]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
14671,0,1,NaN,0,119,NaN,0,0,0,0,0,0,0,1,0,0,0,1,NaN
34008,1,0,0,1,101,0,0,0,0,0,1,0,0,0,0,0,0,1,0
43329,1,0,0,1,101,0,0,0,0,1,1,0,1,0,0,0,0,1,0
63447,0,1,NaN,0,103,NaN,0,0,0,0,1,0,0,0,0,0,NaN,1,NaN
66022,0,0,0,0,101,NaN,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485541,1,1,NaN,0,103,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN
486363,0,1,NaN,0,108,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN
489588,0,0,1,1,102,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
covid_t1[covid_t1.age > 110]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
14671,0,1,NaN,0,119,NaN,0,0,0,0,0,0,0,1,0,0,0,1,NaN
86819,1,1,NaN,0,114,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
105172,0,1,NaN,0,120,NaN,0,0,0,0,0,NaN,0,1,0,0,0,1,NaN
207152,0,1,NaN,0,116,NaN,1,0,0,0,1,0,0,0,0,0,0,1,NaN
210908,0,1,NaN,0,120,NaN,0,0,0,0,0,0,0,0,0,0,0,1,NaN
251406,0,0,0,1,118,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
253200,0,1,NaN,0,120,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN
262617,0,1,NaN,0,120,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN
287439,1,1,NaN,0,117,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN


In [10]:
idade_avancada = 90
covid_idosos = covid_t1[covid_t1.age >= idade_avancada]

# idosos sem doencas
covid_superidosos_mask = (
    (covid_t1.age > idade_avancada) & 
    (covid_t1.pneumonia == 0) & 
    (covid_t1.diabetes == 0) & 
    (covid_t1.copd == 0) &
    (covid_t1.asthma == 0) & 
    (covid_t1.hypertension == 0) & 
    (covid_t1.other_disease == 0) & 
    (covid_t1.cardiovascular == 0) & 
    (covid_t1.renal_chronic == 0)
)

covid_superidosos = covid_t1[covid_superidosos_mask]
covid_superidosos

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
4746,0,1,NaN,0,93,NaN,0,0,0,0,0,0,0,0,0,0,1,1,NaN
7573,1,1,NaN,0,93,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
9662,1,1,NaN,0,94,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN
10559,1,1,NaN,0,91,0,0,0,0,0,0,0,0,0,0,0,1,1,NaN
11214,1,1,NaN,0,91,0,0,0,0,0,0,0,0,1,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494063,0,0,0,0,96,NaN,0,0,0,0,0,0,0,0,0,0,NaN,0,0
494376,0,0,0,0,95,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
494508,1,0,0,0,93,0,0,0,0,0,0,0,0,0,0,0,NaN,0,0


In [11]:
covid_superidosos.describe()

,age
count,360.00000
mean,95.85000
std,5.57299
min,91.00000
25%,92.00000
50%,94.00000
75%,97.25000
max,120.00000


Não parece haver nenhum *outlier* óbvio a respeito de idades avançadas.

Vamos ver se há *outliers* óbvios a respeito de bebês:

In [12]:
# criança grávida?
crianca_gravida_mask = (covid_t1.age < 12) & (covid_t1.pregnancy == 1)
covid_t1[crianca_gravida_mask]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
272278,1,1,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN
318994,1,1,NaN,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,NaN
330427,1,1,NaN,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN
402232,1,1,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN
416157,1,1,NaN,0,11,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN
480636,1,1,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Vamos por fim verificar se há inconsistências:

In [13]:
# homem grávido?
homem_gravido_mask = (covid_t1.sex == 0) & (covid_t1.pregnancy == 1)
covid_t1[homem_gravido_mask]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,


In [14]:
# paciente foi dispensado para casa mas foi intubado?
dispensado_intubado_mask = (covid_t1.patient_type == 1) & (covid_t1.intubed == 1)
covid_t1[dispensado_intubado_mask]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,


#### Resumo

In [15]:
def remover_outliers(df):
    df = df.copy()

    outliers_mask = (
        ((covid_t1.age < 12) & (covid_t1.pregnancy == 1)) | # crianca gravida
        ((covid_t1.sex == 0) & (covid_t1.pregnancy == 1)) | # homem gravido
        ((covid_t1.patient_type == 1) & (covid_t1.intubed == 1))  # paciente dispensado e intubado
    )

    df = df.drop(index = df.index[outliers_mask])

    return df


covid_t2 = (covid_raw
    .pipe(acertar_tipos)
    .pipe(remover_outliers)
)

In [16]:
covid_t2

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
0,0,1,NaN,0,27,NaN,0,0,0,0,0,0,0,0,0,0,0,1,NaN
1,0,1,NaN,0,24,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
2,1,0,0,0,54,0,0,0,0,0,0,0,0,1,0,0,NaN,1,0
3,0,0,0,1,30,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,0
4,1,0,0,0,60,0,1,0,0,0,1,0,1,0,0,0,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499687,0,1,NaN,1,77,NaN,0,0,0,0,0,1,0,0,0,1,0,0,NaN
499688,0,0,1,1,63,NaN,0,0,0,0,1,0,0,1,0,1,0,0,0
499689,1,1,NaN,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


### Análise de duplicatas

Vamos ver se há duplicatas nos dados.

Se todas as colunas forem perfeitamente independentes, há uma chance de...

In [17]:
nunique_cols = covid_t2.apply(lambda col: col.nunique(), axis = 0)
nunique_cols

sex                      2
patient_type             2
intubed                  2
pneumonia                2
age                    120
pregnancy                2
diabetes                 2
copd                     2
asthma                   2
inmsupr                  2
hypertension             2
other_disease            2
cardiovascular           2
obesity                  2
renal_chronic            2
tobacco                  2
contact_other_covid      2
covid_res                2
icu                      2
dtype: int64

In [18]:
total_prodcart = nunique_cols.prod()

print(f'...{1/total_prodcart:.7%} de haver dois pacientes com os mesmos atributos.')
print('\n')
print(f'Em um conjunto com {covid_t2.shape[0]} registros, esperamos que haja {covid_t2.shape[0] / total_prodcart:.3f} entradas duplicadas.')

...0.0000032% de haver dois pacientes com os mesmos atributos.


Em um conjunto com 499686 registros, esperamos que haja 0.016 entradas duplicadas.


Vale lembrar que este valor esperado de 0.016 entradas duplicadas é um piso, visto que intuitivamente temos razão para acreditar a priori que há correlação entre as colunas. Por exemplo, é razoável assumir que há correlação entre a idade e as várias doenças codificadas nos campos. 

In [19]:
covid_t2[covid_t2.duplicated(keep = 'first')]

,sex,patient_type,intubed,pneumonia,age,pregnancy,diabetes,copd,asthma,inmsupr,hypertension,other_disease,cardiovascular,obesity,renal_chronic,tobacco,contact_other_covid,covid_res,icu
id,,,,,,,,,,,,,,,,,,,
22,1,1,NaN,0,45,0,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
27,0,1,NaN,0,40,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
62,0,1,NaN,0,40,NaN,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
70,1,1,NaN,0,33,0,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
76,1,1,NaN,0,38,0,0,0,0,0,0,0,0,0,0,0,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499685,1,1,NaN,0,55,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
499686,1,1,NaN,0,15,0,0,0,0,0,0,0,0,0,0,0,1,0,NaN
499689,1,1,NaN,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


Há 425.262 entradas repetidas. Isso é explicado pela alta correlação entre as colunas, conforme explicitado por testes qui-quadrado.

Relembrando, o teste qui-quadrado testa a relação entre dois conjuntos de dados categóricos:

* $H_0$: conjuntos de dados são independentes
* $H_a$: conjuntos de dados não são independentes

In [20]:
# combinações entre features excluindo 'age'
cols = pd.Index(set(covid_t2.columns) - set(['age']), name = 'cat_feature')

# inicializando o dataframe de combinações entre features
# dataframe é uma matriz quadrada com as linhas e colunas correspondendo aos features
p_values = pd.DataFrame([], 
    columns = cols,
    index = cols
)

# para cada combinação de features ...
for ci in tqdm(p_values.index, desc = 'Features'):
    for cj in p_values.columns:
        if ci == cj:
            # já sabemos que uma combinação com dois features iguais são perfeitamente correlacionadas. rejeitamos o h0 com p_valor 0
            # deixaremos NaN para facilitar sua posterior exclusão da análise
            continue
        
        # ... produz a tabela de contingencia ...
        contingencia = pd.crosstab(        # OBS: essa função dropa valores NaN por default. Não considera-se valores NaN
            covid_t2[ci], covid_t2[cj],
        )

        # ... obtém o p-valor do teste ...
        pvalue = spst.chi2_contingency(contingencia)[1]  

        # ... e salva na célula correspondente
        p_values.at[ci, cj] = pvalue



Features: 100%|██████████| 18/18 [00:19<00:00,  1.10s/it]


In [21]:
p_values

feature,patient_type,obesity,pneumonia,copd,icu,diabetes,tobacco,pregnancy,intubed,hypertension,renal_chronic,inmsupr,cardiovascular,contact_other_covid,other_disease,sex,covid_res,asthma
feature,,,,,,,,,,,,,,,,,,
patient_type,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
obesity,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.067207,0.0,0.0,0.0,0.0
pneumonia,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
copd,0.0,0.0,0.0,NaN,0.007118,0.0,0.0,0.0,0.239392,0.0,0.0,0.0,0.0,0.0,0.0,0.000149,0.000001,0.0
icu,1.0,0.0,0.0,0.007118,NaN,0.00541,0.885849,0.643528,0.0,0.065625,0.002607,0.000579,0.000009,0.001816,0.043507,0.0,0.020822,0.363534
diabetes,0.0,0.0,0.0,0.0,0.00541,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.468996
tobacco,0.0,0.0,0.0,0.0,0.885849,0.0,NaN,0.0,0.192957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000082
pregnancy,0.0,0.0,0.0,0.0,0.643528,0.0,0.0,NaN,0.000001,0.0,0.0,0.106083,0.0,0.0,0.0,1.0,0.252435,0.086063
intubed,1.0,0.0,0.0,0.239392,0.0,0.0,0.192957,0.000001,NaN,0.000004,0.498287,0.492352,0.000552,0.0,0.014665,0.0,0.0,0.000047


Vamos quantificar o número de features dependentes:

In [22]:
# aplicando a correção de bonferroni, que divide o nível de significancia desejado
# pelo número de testes (features ** 2 menos uma vez a quantidade de features)
alpha = 0.05
alpha_bonferroni = alpha / (cols.shape[0] ** 2 - cols.shape[0])

p_value_long = p_values.reset_index().melt(id_vars = ['feature'], var_name = 'feature2', value_name = 'p_value')
p_value_long.dropna(inplace = True)

# criando a coluna de rejeição/não-rejeição da hipótese nula
p_value_long['H0'] = np.where(p_value_long['p_value'] < alpha_bonferroni, False, True)
p_value_long

,feature,feature2,p_value,H0
1,obesity,patient_type,0.0,False
2,pneumonia,patient_type,0.0,False
3,copd,patient_type,0.0,False
4,icu,patient_type,1.0,True
5,diabetes,patient_type,0.0,False
...,...,...,...,...
318,cardiovascular,asthma,0.0,False
319,contact_other_covid,asthma,0.0,False
320,other_disease,asthma,0.0,False
321,sex,asthma,0.0,False


In [23]:
p_value_long['H0'].value_counts(normalize = True)

False    0.823529
True     0.176471
Name: H0, dtype: float64

Em resumo, podemos rejeitar a hipótese de independência em 82% das combinações de *features*. Isso aumenta significativamente o número esperado de duplicatas.

Concluímos que, como essas duplicatas não estão fora da expectativa razoável, não é válido excluir as duplicatas. Não há transformações a fazer nessa seção.

In [24]:
def remover_duplicatas(df):
    return df

covid_t3 = (covid_raw
    .pipe(acertar_tipos)
    .pipe(remover_outliers)
    .pipe(remover_duplicatas)
)

### Análise de valores faltantes

Por fim, analisaremos os valores faltantes.

In [25]:
print('Valores faltantes:')
with pd.option_context('display.float_format', '{:.1%}'.format):
    display(covid_t3.isna().sum() / (covid_t3.count() + covid_t3.isna().sum()))

Valores faltantes:


sex                    0.0%
patient_type           0.0%
intubed               78.5%
pneumonia              0.0%
age                    0.0%
pregnancy             50.9%
diabetes               0.3%
copd                   0.3%
asthma                 0.3%
inmsupr                0.3%
hypertension           0.3%
other_disease          0.4%
cardiovascular         0.3%
obesity                0.3%
renal_chronic          0.3%
tobacco                0.3%
contact_other_covid   30.8%
covid_res              0.0%
icu                   78.5%
dtype: float64

Em resumo:

* a maioria dos valores nos campos `intubed` (se a pessoa foi intubada ou não) `icu` (se a pessoa foi transferida para a UTI ou não) e `pregnancy` (se a pessoa está grávida ou não) - 79%, 79% e 51%, respectivamente - estão faltando;
* uma parcela relevante (31%) dos valores no campo `contact_other_covid` (se a pessoa entrou em contato com outras pessoas que tiveram resultado positivo em teste para COVID) está faltando; e
* nos outros campos, a parcela de valores faltantes é ínfima.

Logo, **adotaremos como estratégia**:
* **descartar os campos `intubed`, `icu` e `pregnancy`** da análise, visto que o preenchimento destes campos não é confiável e portanto não são úteis para construção de um modelo preditivo que dependa destes campos;
* **codificar os valores faltantes do campo `contact_other_covid` como um valor a parte**. Nesse caso, nada a fazer nesta seção: processaremos os valores faltantes desse campo na hora da construção do modelo; 
  * É possível que, ao construir o modelo, verifiquemos que esse campo confira ao mesmo baixo poder preditivo. Neste caso, nossa estratégia para lidar com valores faltantes nesse campo pode mudar
* **eliminar da análise os valores faltantes no resto dos campos**.

In [26]:
def tratar_valores_faltantes(df):
    df = df.copy()

    # dropando colunas 'intubed', 'icu', 'pregnancy'
    df = df.drop(columns = ['intubed', 'icu', 'pregnancy'])
    
    # dropando missings das outras colunas
    df = df.dropna(subset = set(df.columns) - set(['contact_other_covid']))

    # agora que não há mais dados faltantes na coluna de 'age', podemos reverter para o tipo 'int8' do numpy
    # (ao invés do tipo 'Int8' do pandas, que aceita NaN)
    df['age'] = df['age'].astype('int8')

    # codificando NaN da coluna 'contact_other_covid' como -1
    # for c in ['contact_other_covid',]:
    #     df[c] = df[c].cat.codes
    #     df[c] = df[c].astype('category')

    return df

covid = (covid_raw
    .pipe(acertar_tipos)
    .pipe(remover_outliers)
    .pipe(remover_duplicatas)
    .pipe(tratar_valores_faltantes)
)

### Dados finais

In [27]:
covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496285 entries, 0 to 499691
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   sex                  496285 non-null  category
 1   patient_type         496285 non-null  category
 2   pneumonia            496285 non-null  category
 3   age                  496285 non-null  int8    
 4   diabetes             496285 non-null  category
 5   copd                 496285 non-null  category
 6   asthma               496285 non-null  category
 7   inmsupr              496285 non-null  category
 8   hypertension         496285 non-null  category
 9   other_disease        496285 non-null  category
 10  cardiovascular       496285 non-null  category
 11  obesity              496285 non-null  category
 12  renal_chronic        496285 non-null  category
 13  tobacco              496285 non-null  category
 14  contact_other_covid  343718 non-null  category
 15  

In [28]:
covid.describe()

,age
count,496285.000000
mean,42.513425
std,16.622971
min,0.000000
25%,31.000000
50%,41.000000
75%,53.000000
max,120.000000


In [29]:
# categorias

covid.select_dtypes(include = 'category').apply(lambda x: x.cat.categories).transpose().apply(np.array, axis = 1)

sex                    [0, 1]
patient_type           [0, 1]
pneumonia              [0, 1]
diabetes               [0, 1]
copd                   [0, 1]
asthma                 [0, 1]
inmsupr                [0, 1]
hypertension           [0, 1]
other_disease          [0, 1]
cardiovascular         [0, 1]
obesity                [0, 1]
renal_chronic          [0, 1]
tobacco                [0, 1]
contact_other_covid    [0, 1]
covid_res              [0, 1]
dtype: object

## Exportação para subsequente uso

Vamos por fim salvar os dados finais com um formato próprio do `pandas` que economiza no processamento subsequente:

In [46]:
with open(r'../data/covid_limpo.df', 'wb') as covidfile:
    pickler = pickle.Pickler(file = covidfile)
    pickler.dump(covid)